In [1]:
# putting the contents of a text file into a panda dataframe

In [2]:
import pandas as pd

In [3]:
cd death-lessons_old/

[Errno 2] No such file or directory: 'death-lessons_old/'
/media/drcjar/Expansion/death-lessons_old


In [4]:
df = pd.read_csv('death_lessons_dataframe.csv')

In [5]:
files = !ls downloads/

In [6]:
cd downloads

/media/drcjar/Expansion/death-lessons_old/downloads


In [7]:
text_reports = {}

for f in files:
    try:
        with open(f, 'r') as reader:
            text_reports[f] = reader.read()
    except:
        continue

In [8]:
df1 = pd.DataFrame.from_dict(text_reports, orient='index', columns=['text'])
df1 = df1.reset_index()
df1.columns = ['filename', 'text']

In [9]:
# find reference in our dataframe and in our filenames
df['ref'] = df['ref'].str.extract(r'(\d{4}-\d{4})') # extract reference if there is one
df1['ref'] = df1['filename'].str.extract(r'(\d{4}-\d{4})') # extract reference if there is one

In [10]:
df.ref.isnull().value_counts()

False    3819
True      116
Name: ref, dtype: int64

In [11]:
df1.ref.isnull().value_counts()

False    19573
True      1112
Name: ref, dtype: int64

In [12]:
# discard nulls for now
df = df[~df.ref.isnull()]
df1 = df1[~df1.ref.isnull()]

In [13]:
# merge on ref
df = pd.merge(df,df1, on='ref')

In [14]:
df.ref.isnull().value_counts()

False    19331
Name: ref, dtype: int64

In [15]:
df['short_name'] = df['name_of_deceased'].str.replace('Deceased name: ','')

In [16]:
df['short_name'] = df['short_name'].str.lower()

In [17]:
df['short_name'] = df['short_name'].str.replace(' ','-')

In [18]:
df['short_name'] = df['short_name'].str.strip()

In [19]:
df['short_name']

0                       <p>dominic-noble</p>
1                       <p>dominic-noble</p>
2                       <p>daniel-xavier</p>
3                      <p>shona-campbell</p>
4                        <p>paul-meadows</p>
                        ...                 
19326               <p>annie-rose-gibson</p>
19327               <p>annie-rose-gibson</p>
19328               <p>annie-rose-gibson</p>
19329    <p>derek-edward-bartlett-twivey</p>
19330                   <p>phillip-pratt</p>
Name: short_name, Length: 19331, dtype: object

In [20]:
# https://www.judiciary.uk/prevention-of-future-death-reports/constance-pridmore/

# https://www.judiciary.uk/prevention-of-future-death-reports/andrew-brown-prevention-of-future-deaths-report/

In [21]:
df['url'] = 'https://www.judiciary.uk/prevention-of-future-death-reports/' + df['short_name'].astype(str) + '/'

In [22]:
# clean up data (still contains html tags and duplicates) ? use soup.get_text() earlier
# decide what to display and how
# style with css

In [23]:
# remove html tags
df['date_of_report'] = df['date_of_report'].str.replace('<[^<]+?>', '')
df['name_of_deceased'] = df['name_of_deceased'].str.replace('<[^<]+?>', '')
df['coroner_name'] = df['coroner_name'].str.replace('<[^<]+?>', '')
df['coroner_area'] = df['coroner_area'].str.replace('<[^<]+?>', '')
df['url'] = df['url'].str.replace('<[^<]+?>', '')
# remove duplicates
df = df.drop_duplicates(subset='filename')

In [24]:
del df['short_name']

In [25]:
df['url'][0]

'https://www.judiciary.uk/prevention-of-future-death-reports/dominic-noble/'

In [26]:
df.to_json("data.json", lines=True, orient="records")

In [27]:
df['url'][0]

'https://www.judiciary.uk/prevention-of-future-death-reports/dominic-noble/'

In [28]:
df['year_of_report'] = df['ref'].str[:4].astype(int)

In [29]:
df.ref

0        2022-0204
1        2022-0204
2        2022-0203
3        2022-0202
4        2022-0201
           ...    
19326    2013-0171
19327    2013-0171
19328    2013-0171
19329    2013-0175
19330    2013-0174
Name: ref, Length: 18451, dtype: object

In [30]:
df.year_of_report.max()

2022

In [31]:
df.year_of_report.min()

2013

In [32]:
df.name_of_deceased.nunique()

3655

In [33]:
df[df.year_of_report == 2013]

,person,date_of_report,ref,name_of_deceased,coroner_name,coroner_area,category,filename,text,url,year_of_report
18062,nilsson-2013-0332,Date of report: 2 December 2013,2013-0332,Deceased name: Karl Olof Nilsson,Coroners name: Caroline Sarah Sumeray,Coroners Area: West Yorkshire (Western),<STRONG>CATEGORY</STRONG>: ROAD (HIGHWAYS SAFETY),Nilsson+2013-0332.pdf.txt,\n \n\n \n \n \n \n\n \n\n \n\n \n\n \n\n \n\...,https://www.judiciary.uk/prevention-of-future-...,2013
18063,barton-2013-0330,Date of report: 6 December 2013,2013-0330,Deceased name: Keith Barton,Coroners name: Patricia Harding,Coroners Area: Mid Kent and Medway,<STRONG>CATEGORY</STRONG>: CARE HOME HEALTH,2013-0330-Response-by-Life-Style-Care_page0.txt,LY-TIHK -cv14 UStD>S From: lo? Seu rey4o14u Pa...,https://www.judiciary.uk/prevention-of-future-...,2013
18064,barton-2013-0330,Date of report: 6 December 2013,2013-0330,Deceased name: Keith Barton,Coroners name: Patricia Harding,Coroners Area: Mid Kent and Medway,<STRONG>CATEGORY</STRONG>: CARE HOME HEALTH,2013-0330-Response-by-Life-Style-Care_page1.txt,\n\n{ met with SALT on 17 January 2014 in ord...,https://www.judiciary.uk/prevention-of-future-...,2013
18065,barton-2013-0330,Date of report: 6 December 2013,2013-0330,Deceased name: Keith Barton,Coroners name: Patricia Harding,Coroners Area: Mid Kent and Medway,<STRONG>CATEGORY</STRONG>: CARE HOME HEALTH,2013-0330-Response-by-Life-Style-Care_page2.txt,ee ee 10; SucY fraS+b 1S Page:4~4\n\n \n\n| bo...,https://www.judiciary.uk/prevention-of-future-...,2013
18066,barton-2013-0330,Date of report: 6 December 2013,2013-0330,Deceased name: Keith Barton,Coroners name: Patricia Harding,Coroners Area: Mid Kent and Medway,<STRONG>CATEGORY</STRONG>: CARE HOME HEALTH,2013-0330-Response-by-Life-Style-Care.pdf.txt,,https://www.judiciary.uk/prevention-of-future-...,2013
...,...,...,...,...,...,...,...,...,...,...,...
19326,annie-rose-gibson,Date of report: 1 August 2013,2013-0171,Deceased name: Annie Rose Gibson,Coroners name: David Hinchcliff,Coroners Area: West Yorkshire (East),CATEGORY: COMMUNITY HEALTH CARE AND EMERGENCY ...,Gibson-2013-01711_page1.txt,a daily basis by her family and home carers fr...,https://www.judiciary.uk/prevention-of-future-...,2013
19327,annie-rose-gibson,Date of report: 1 August 2013,2013-0171,Deceased name: Annie Rose Gibson,Coroners name: David Hinchcliff,Coroners Area: West Yorkshire (East),CATEGORY: COMMUNITY HEALTH CARE AND EMERGENCY ...,Gibson-2013-01711_page2.txt,| enclose for a assistance the statements of y...,https://www.judiciary.uk/prevention-of-future-...,2013
19328,annie-rose-gibson,Date of report: 1 August 2013,2013-0171,Deceased name: Annie Rose Gibson,Coroners name: David Hinchcliff,Coroners Area: West Yorkshire (East),CATEGORY: COMMUNITY HEALTH CARE AND EMERGENCY ...,Gibson-2013-01711.pdf.txt,,https://www.judiciary.uk/prevention-of-future-...,2013
19329,derek-edward-bartlett-twivey,Date of report: 30 July 2013,2013-0175,Deceased name: Derek Edward Bartlett Twivey,Coroners name: Elisabeth Bussey-Jones,Coroners Area: West Sussex,CATEGORY: COMMUNITY HEALTH CARE AND EMERGENCY ...,Twivey-2013-0175.pdf.txt,Penelope A Schofield \nSenior Coroner \nCounty...,https://www.judiciary.uk/prevention-of-future-...,2013


In [34]:
df.name_of_deceased.value_counts()

Deceased name: Terence O’Connell              45
Deceased name: Elena Wells                    37
Deceased name: Vhari Ingall                   36
Deceased name: Alex Kelly                     34
Deceased name: Santosh Muthiah                33
                                              ..
Deceased name: Robert Anstice                  1
Deceased name: Herbert Chandler                1
Deceased name: Peter Patrick Adrian Barnes     1
Deceased name: Thaker Hafid                    1
Deceased name: Jane Dyson Gabbitas             1
Name: name_of_deceased, Length: 3655, dtype: int64

In [35]:
df.columns

Index(['person', 'date_of_report', 'ref', 'name_of_deceased', 'coroner_name',
       'coroner_area', 'category', 'filename', 'text', 'url',
       'year_of_report'],
      dtype='object')

In [36]:
jam = df[df.text.str.upper().str.contains('CHEST DRAIN')]

In [37]:
jam.ref.unique()

array(['2021-0276', '2020-0281', '2020-0145', '2019-0343', '2017-0013',
       '2016-0258', '2016-0491', '2015-0135', '2014-0570', '2014-0355',
       '2013-0327', '2013-0308', '2013-0244', '2013-0271'], dtype=object)